In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# pull in ODE def & solver
from three_body import three_body_ode, solve

## Example Three Body Computation
We'll find an approximate the solution using the value of $R_0$ below

In [ ]:

R0 = np.array([
    [-1.2, -0.80],
    [ 1.0, -0.83],
    [ 1.1,  0.80]
])

sol = solve(R0)
sol


We'll now plot the computed trajectories, indicating the initial positions of each body with a red cross symbol.

In [ ]:

plt.title(f'$R_0 = [{" ,".join(map(str,R0.flatten()))}]$')
for ii in range(3):
    plt.plot(sol[:,ii,0],sol[:,ii,1])
    
plt.scatter(
    sol[0,:,0], sol[0,:,1],
    c='r', marker='+', s=30
)


## Specializing On A Familty of Initial Values
For the purposes of this example, we're interested in collecting information about the behavior of solutions to the three body problem (in 2D) when the initial positions ($r_1$, $r_2$ and $r_3$) are chosen so as to satisfy:
$$
\begin{align}
 r_1 & \sim \mathcal{N}_2\left([-1,-1],\ 0.25\cdot I_2\right) \\
 r_2 & \sim \mathcal{N}_2\left([\ \ \ 1,-1],\ 0.25 \cdot I_2\right) \\
 r_3 & \sim \mathcal{N}_2\left([\ \ \ 1, \ \ \ 1],\ 0.25\cdot I_2\right) \\
\end{align}
$$


Translating this to something tangible, it can be understood as requiring that we only compute solutions with initial values chosen randomly using a function like:

In [ ]:

random_position = lambda : np.array([
      [np.random.normal(-1, 0.25), np.random.normal(-1, 0.25)],
      [np.random.normal( 1, 0.25), np.random.normal(-1, 0.25)],
      [np.random.normal( 1, 0.25), np.random.normal( 1, 0.25)]
  ])


However, because of the way pseudo-random numbers are generated by `numpy`; we can't simply put the code from the above cell in `three_body.py` then run it a bunch of times, since this would result in extremely poor sampling/parameter-selection

### Workaround
We have to be deliberate with how we choose and distribute the initial positions. There are several ways we can accomplish this vague task, however in this example we'll only concern ourselves with one.

In this case, what we are going to do is create one very large sample/parameter-list which satisfies the conditions we are looking for, and then use `SLURM` to schedule the computations.

In [ ]:


# sample-size/number of parameters
N = 5000

# generating and sorting the sample/parameters
with open('parameters.txt', 'w') as params_file:
    for _ in range(N):
        # observe a random positiion 
        initial_pos = random_position().flatten()
        # store in `params_file` for use later
        print(' '.join(['{:0.5f}']*6).format(*initial_pos), file=params_file)


### Where Job Arrays Come In
We can index the `parameters.txt` using line numbers, and this is fairly standard to do:

In [ ]:
!sed -n 2134p "parameters.txt"

What's important about this is that SLURM Job Arrays provide us a sane mechanism we can use to index a collection of jobs.

It does this by making available the `SLURM_ARRAY_TASK_ID` environment variable inside of each executing job for the array. 

In [ ]:
%%bash
#!/bin/bash
### Job Array:
#SBATCH --array 1-5000%128

### Array Task Parameters:
#SBATCH --job-name  "Three-Body Task" # display name
#SBATCH --output    "logs/out.%a.log"   # where to log terminal output 
#SBATCH --error     "logs/err.%a.log"   #  .. and error messages
#SBATCH --open-mode append

# Resources required
#SBATCH --ntasks 1          # number of tasks we'll perform
#SBATCH --cpus-per-task 1   # num. cpus each task will require
#SBATCH --mem-per-cpu 1024  # memory required per cpu (in megabytes)

### Script To Execute:

# create places to hold results
mkdir -p "logs" "imgs"

# load deps (OR ALTERNATIVELY: Activate virtual environment.)
module load GCCcore Python intel SciPy-bundle

# use the SLURM task id to select parameters from list.
param_args=$(sed -n ${SLURM_ARRAY_TASK_ID}p "parameters.txt")

# run code with selected parameters
python three_body.py $param_args 

